# Week 1 -- Basics of an Ideal Gas

`jupyter` notebooks allow you to interleave text, code, and results into one document. For this week's project, I've started a notebook for you with some code sketches already filled in. Please make sure you can run the code snippets that are already here, and then expand on it to complete the rest of the project. Adapted from Zach Berta-Thompson.

In [ ]:
# gain access to numerical tools
import numpy as np

# gain access to plotting tools 
import matplotlib.pyplot as plt 
# make plots show up in the notebook
%matplotlib inline
# set up bigger default plots
plt.matplotlib.rcParams['figure.figsize'] = (10,7)
plt.matplotlib.rcParams['font.size'] = 12

## Using units

Here, we'll be using the astropy "units" module to keep track of units for us. This is extremely convenient because it allows us to convert between units, and keep track of our units, automatically. From personal experience, there can be huge problems if you get this wrong. As a warning, you do need to make sure you have the units you want before using the value somewhere that you aren't keeping track of units. Here's an example:

In [ ]:
# gain access to astropy units
from astropy import units, constants

In [ ]:
myvelocity = 1e-5*constants.c  # use the speed of light stored in astropy.constants
print(myvelocity)
print(myvelocity.to('cm/s'))   # change the units to cm/s

mydistance = myvelocity*(5*units.min)
print(mydistance)# multiply by 5 min
# hmmm, units of "min/s" is kind of silly
# either of these simplifies the units for you
print(mydistance.decompose(), "or", mydistance.to('m')) 

In [ ]:
plt.scatter(myvelocity, mydistance)

xunit_string = myvelocity.unit.to_string('latex_inline')
yunit_string = mydistance.decompose().unit.to_string('latex_inline')
plt.ylabel('Distance ({})'.format(yunit_string))
plt.xlabel('Distance ({})'.format(xunit_string))

### Q1: An Ideal Gas

Earth's atmospheric pressure at sea level is $1.01 \times 10^5~Pa$ (where $1~Pa = 1~N/m^2 = 10^{-5}~bar$). At room temperature ($293 K$) and this pressure, how much volume would be filled by a little cloud of $10^6~ N_2$ molecules?

In [ ]:
# For the sake of getting use to Python and jupyter notebooks,
# please use this code cell to perform your calculation.
# (Be sure to indicate the units associated with your answer).

N = int(1e6)                  # total number of molecules
pressure = 1.01e5*units.Pa    # in units of Pa
temperature = 293*units.K     # in units of K
k_B = constants.k_B           # Boltzmann's constant

# ...

### Q2: Speeds in Gases 

Let's play around a little with these $10^6~N_2$ atoms, and the speeds at which they're moving. The Maxwell-Boltzmann distribution gives the probability for a particle in an ideal gas to have the $x$-component of its *velocity* fall between $v_x$ and $v_x + dv_x$ is written as 

$$ f(v_x)dv_x = \left(\frac{m}{2\pi k_B T}\right)^{1/2}\exp \left(\frac{-mv_x^2}{2 k_B T} \right)dv_x $$

where $m$ is the mass of the particle, $T$ is the temperature of the gas, and $k_B = 1.38 \times 10^{-23}~J~K^{-1}$ is Boltzmann's constant. If we define the quantity $\sigma = \sqrt{k_B T/m}$, then the above expression can be rewritten as 

$$ f(v_x)dv_x = \frac{1}{\sqrt{2\pi} \sigma}\exp \left(\frac{-v_x^2}{2 \sigma^2} \right)dv_x $$

which is precisely the equation for a Gaussian or "normal" probability distribution, centered at $v_x = 0$ and with a width of $\sigma$. 

Let's use Python to calculate $\sigma$ for simulate the $x$-component of the velocities for $10^6$ imaginary $N_2$ molecules, and then we'll create a `numpy` array that contains $10^6$ random numbers drawn from this probability distribution. 

In [ ]:
# define our values
temperature = 293*units.K         # the temperature, in units of K
k_B = constants.k_B               # Boltzmann's constant, in units of J/K
mass = 28*units.Dalton            # the mass of an N2 molecule, in amu (or Daltons)

# calculate sigma from these (check on paper that the units make sense!)
sigma = np.sqrt( (k_B*temperature/mass).decompose() )   # in units of m/s

# draw a 1e5 random numbers from a Gaussian ("normal") distribution, 
# centered at vx=0 and with a width of sigma
# note these numpy functions can't use numbers with units
vx = np.random.normal(0,sigma.value,N) #strip sigma of its units

Now, let's see how frequently particular values of $v_x$ occur, by plotting a histogram. Using the `density=True` option in `plt.hist` means this distribution will be plotted as a probability density (which is a function that integrates to 1).

In [ ]:
# let's plot a histogram of those values
plt.hist(vx, bins=50, density=True)

# always be sure to label your axes
plt.xlabel('$x$-component of Velocity (m/s)')
plt.ylabel('Probability Density')
plt.title('Distribution of Velocities in the $x$ Direction')


# now, let's overplot the analytic shape of the probability distribution
#  first, we create a fine grid of vx values
vx_axis = np.linspace(-1500, 1500, 1000)*units.m/units.s
#  then, we use the above equation to calculate the probability
probability = 1/np.sqrt(2*np.pi)/sigma*np.exp(-vx_axis**2/2/sigma**2)
#  finally, we plot these x and y values as an orange line
plt.plot(vx_axis, probability, linewidth=5, color='darkorange', alpha=0.5)

# we can plot the expected average value for vx
plt.axvline(0, color='gray', label='expected average velocity')

# we can plot the actual average value of vx, measured from our simulation
vx_average = np.mean(vx)
plt.axvline(vx_average, linestyle='--', color='black', label='actual average velocity')

# add a legend, including the labels that were assigned to each line
plt.legend(frameon=False);

Hey, cool! The histogram of random numbers that we created with Python follows the shape of the distribution we draw them from. Maybe that's not so surprising, but it's a useful test! Now, let's expand this to three dimensions of motion, and calculate a few more quantities.



+ Create two more arrays for $v_y$ and $v_z$ in the same way you did for $v_x$, assuming the motion in the three directions is totally independent. Make a scatter plot showing the components $v_x$ vs $v_y$. (Be sure to label your axes). (This might take a moment depending on how fast your computer is, feel free to change N to a smaller number while you are testing things out!)


+ Plot a histogram of the kinetic energy of the molecules, $E_{kinetic} = \frac{1}{2}mv^2$, where $v^2 = v_x^2 + v_y^2 + v_z^2$. Be sure to label your axes. 

+ The average kinetic energy of the molecules' translational motion should be $\frac{3}{2}k_B T$. Show that this statement agrees with the average kinetic energy  you measure for the particles in your simulation.

Before plotting histograms, you'll need to get rid of your units. That is, instead of plotting the variable `kineticenergy`, plot `kineticenergy.to('Joule').value`.

+ Plot a histogram of $s = \sqrt{v^2}$, where $s$ indicates the speed of the particles (the magnitude of their 3D velocity vectors). 

+ Indicate with vertical lines on your plot the gravitational escape speed of the planet Earth (11.2 km/s) and the approximate escape speed of the comet 67P/Churyumov–Gerasimenko (1 m/s). 


+ Briefly discuss the implications for the long-term persistence of an atmosphere on these two bodies.

<font color='seagreen'>

Your brief discussion (double click to edit):